# Preparing demo data

In this notebook, we read in the Mendel Option 29 (Ped-GWAS) data, introduce some missing genotypes, and then save the modified data set for demonstration purpose.

Following shell commands assumes MacOS or Linux environment. Julia commands should run regardless of OS.

## Data files

We start from the following 3 files from [Mendel Option 29 (Ped-GWAS) example](https://www.genetics.ucla.edu/software/Mendel_current_doc.pdf#page=294). 

In [1]:
;ls -l Ped29c.in SNP_data29a.bin SNP_def29a.in

-rw-rw-r--  1 huazhou  staff     18868 Aug 21  2016 Ped29c.in
-rw-rw-r--  1 huazhou  staff  13416476 Aug 21  2016 SNP_data29a.bin
-rw-rw-r--  1 huazhou  staff  11990931 Aug 21  2016 SNP_def29a.in


Because `SnpArray` function requires input file name ending in `.bed`, we create a symbolic link `SNP_data29a.bed` to `SNP_data29a.bin`.

In [2]:
;ln -s ./SNP_data29a.bin ./SNP_data29a.bed

In [3]:
;ls -l Ped29c.in SNP_def29a.in SNP_data29a.*

Stacktrace:
 [1] depwarn(::String, ::Symbol) at /Applications/Julia-0.6.app/Contents/Resources/julia/lib/julia/sys.dylib:?
 [2] warn_shell_special(::String) at /Applications/Julia-0.6.app/Contents/Resources/julia/lib/julia/sys.dylib:?
 [3] #shell_parse#236(::String, ::Function, ::String, ::Bool) at /Applications/Julia-0.6.app/Contents/Resources/julia/lib/julia/sys.dylib:?
 [4] (::Base.#kw##shell_parse)(::Array{Any,1}, ::Base.#shell_parse, ::String, ::Bool) at /Applications/Julia-0.6.app/Contents/Resources/julia/lib/julia/sys.dylib:? (repeats 2 times)
 [5] @cmd(::ANY

-rw-rw-r--  1 huazhou  staff     18868 Aug 21  2016 Ped29c.in
lrwxr-xr-x  1 huazhou  staff        17 Sep 12 09:41 SNP_data29a.bed -> ./SNP_data29a.bin
-rw-rw-r--  1 huazhou  staff  13416476 Aug 21  2016 SNP_data29a.bin
-rw-rw-r--  1 huazhou  staff  11990931 Aug 21  2016 SNP_def29a.in


) at /Applications/Julia-0.6.app/Contents/Resources/julia/lib/julia/sys.dylib:?
 [6] include_string(::String, ::String) at /Applications/Julia-0.6.app/Contents/Resources/julia/lib/julia/sys.dylib:?
 [7] include_string(::Module, ::String, ::String) at /Users/huazhou/.julia/v0.6/Compat/src/Compat.jl:407
 [8] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/huazhou/.julia/v0.6/IJulia/src/execute_request.jl:154
 [9] eventloop(::ZMQ.Socket) at /Users/huazhou/.julia/v0.6/IJulia/src/eventloop.jl:8
 [10] (::IJulia.##14#17)() at ./task.jl:335
while loading In[3], in expression starting on line 1


## Read in Mendel Option 29 data

Take a look at the first 10 lines of the pedigree file.

In [4]:
;head Ped29c.in

  1       ,  16      ,          ,          ,  F       ,          ,  30.20564,   9.24210,
  1       ,  8228    ,          ,          ,  F       ,          ,  35.82143,  15.27458,
  1       ,  17008   ,          ,          ,  M       ,          ,  36.05298,  19.50496,
  1       ,  9218    ,  17008   ,  16      ,  M       ,          ,  38.96351,  18.98575,
  1       ,  3226    ,  9218    ,  8228    ,  F       ,          ,  33.73911,  21.10412,
  2       ,  29      ,          ,          ,  F       ,          ,  34.88835,  19.01142,
  2       ,  2294    ,          ,          ,  M       ,          ,  37.70105,  19.16556,
  2       ,  3416    ,          ,          ,  M       ,          ,  45.13171,  19.84088,
  2       ,  17893   ,  2294    ,  29      ,  F       ,          ,  35.15599,  14.14228,
  2       ,  6952    ,  3416    ,  17893   ,  M       ,          ,  42.45136,  19.92713,


Read in the pedigree file.

In [5]:
# columns are: :famid, :id, :moid, :faid, :sex, :twin, :simtrait1, :simtrait2, :group
ped29c = readcsv("Ped29c.in", Any; header = false)

212×9 Array{Any,2}:
     1     16       "          "  …  "          "  30.2056   9.2421  ""
     1   8228       "          "     "          "  35.8214  15.2746  ""
     1  17008       "          "     "          "  36.053   19.505   ""
     1   9218  17008                 "          "  38.9635  18.9857  ""
     1   3226   9218                 "          "  33.7391  21.1041  ""
     2     29       "          "  …  "          "  34.8884  19.0114  ""
     2   2294       "          "     "          "  37.7011  19.1656  ""
     2   3416       "          "     "          "  45.1317  19.8409  ""
     2  17893   2294                 "          "  35.156   14.1423  ""
     2   6952   3416                 "          "  42.4514  19.9271  ""
     2  14695   2294              …  "          "  35.6426  17.4191  ""
     2   6790   2294                 "          "  40.6344  23.6845  ""
     2   3916   2294                 "          "  34.8618  16.8684  ""
     ⋮                            ⋱  ⋮      

Take a look at the first 10 lines of the SNP definition file.

In [6]:
;head SNP_def29a.in

    3.00  = FILE FORMAT VERSION NUMBER.
 
rs3020701       ,19,           90974,   1,   2
rs56343121      ,19,           91106,   1,   2
rs143501051     ,19,           93542,   1,   2
rs56182540      ,19,           95981,   1,   2
rs7260412       ,19,          105021,   1,   2
rs11669393      ,19,          107866,   1,   2
rs181646587     ,19,          107894,   1,   2
rs8106297       ,19,          107958,   1,   2


Read in the SNP definition file, skipping the first 2 lines.

In [7]:
# columns are: :snpid, :chrom, :pos, :allele1, :allele2, :groupname
snpdef29c = readcsv("SNP_def29a.in", Any; skipstart = 2, header = false)

253141×6 Array{Any,2}:
 "rs3020701       "  19     90974  1  2  ""
 "rs56343121      "  19     91106  1  2  ""
 "rs143501051     "  19     93542  1  2  ""
 "rs56182540      "  19     95981  1  2  ""
 "rs7260412       "  19    105021  1  2  ""
 "rs11669393      "  19    107866  1  2  ""
 "rs181646587     "  19    107894  1  2  ""
 "rs8106297       "  19    107958  1  2  ""
 "rs8106302       "  19    107962  1  2  ""
 "rs183568620     "  19    107987  1  2  ""
 "rs186451972     "  19    108003  1  2  ""
 "rs189699222     "  19    108032  1  2  ""
 "rs182902214     "  19    108090  1  2  ""
 ⋮                                       ⋮ 
 "rs188169422     "  19  59116080  1  2  ""
 "rs144587467     "  19  59117729  1  2  ""
 "rs139879509     "  19  59117949  1  2  ""
 "rs143250448     "  19  59117982  1  2  ""
 "rs145384750     "  19  59118028  1  2  ""
 "rs149215836     "  19  59118040  1  2  ""
 "rs139221927     "  19  59118044  1  2  ""
 "rs181848453     "  19  59118114  1  2  ""
 "rs13831

Read in the SNP binary file using the SnpArray.jl package.

In [8]:
using SnpArrays

snpbin29a = SnpArray("SNP_data29a"; people = size(ped29c, 1), snps = size(snpdef29c, 1))

INFO: v1.0 BED file detected


212×253141 SnpArrays.SnpArray{2}:
 (true, true)  (true, true)   …  (true, true)    (false, false)
 (true, true)  (true, true)      (true, true)    (false, false)
 (true, true)  (true, true)      (true, true)    (false, false)
 (true, true)  (true, true)      (true, true)    (false, false)
 (true, true)  (true, true)      (true, true)    (false, false)
 (true, true)  (true, true)   …  (false, false)  (true, true)  
 (true, true)  (false, true)     (false, false)  (true, true)  
 (true, true)  (true, true)      (false, false)  (true, true)  
 (true, true)  (false, true)     (false, false)  (true, true)  
 (true, true)  (false, true)     (false, false)  (true, true)  
 (true, true)  (false, true)  …  (false, false)  (true, true)  
 (true, true)  (true, true)      (false, false)  (true, true)  
 (true, true)  (false, true)     (false, false)  (true, true)  
 ⋮                            ⋱                  ⋮             
 (true, true)  (true, true)   …  (true, true)    (false, false)
 (true

## Introduce missingness in genotypes

Let us make about 1% of the genotypes missing.

In [9]:
srand(123) # seed
missing_proportion = 0.01
snpbin29a_missing = copy(snpbin29a)
for i in eachindex(snpbin29a)
    rand() < missing_proportion && (snpbin29a_missing[i] = (true, false))
end

Check how many missing genotypes are introduced.

In [10]:
maf, minor_allele, missings_by_snp, missings_by_person = summarize(snpbin29a_missing)
println("Number of missing genotypes = $(sum(missings_by_snp))")
println("Proportion of missing genotypes = $(sum(missings_by_snp) / length(snpbin29a_missing))")

Number of missing genotypes = 536722
Proportion of missing genotypes = 0.010001175420693651


## Form SnpData and save as Plink files

In [11]:
people = size(ped29c, 1)
snps = size(snpdef29c, 1)
personid = map(string, ped29c[:, 1])
snpid = map(x -> strip(string(x)), snpdef29c[:, 1])
chromosome = map(string, snpdef29c[:, 2])
genetic_distance = zeros(snps)
basepairs = convert(Vector{Int}, snpdef29c[:, 3])
allele1 = map(string, snpdef29c[:, 4])
allele2 = map(string, snpdef29c[:, 5])
snpmatrix = snpbin29a_missing
snpdata29c_missing = SnpData(people, snps, personid, snpid, chromosome, genetic_distance, basepairs, 
    allele1, allele2, maf, minor_allele, snpmatrix, missings_by_person, missings_by_snp)

SnpArrays.SnpData(212, 253141, AbstractString["1", "1", "1", "1", "1", "2", "2", "2", "2", "2"  …  "31", "31", "10006", "10008", "10014", "10027", "10029", "10033", "10040", "10045"], AbstractString["rs3020701", "rs56343121", "rs143501051", "rs56182540", "rs7260412", "rs11669393", "rs181646587", "rs8106297", "rs8106302", "rs183568620"  …  "rs139879509", "rs143250448", "rs145384750", "rs149215836", "rs139221927", "rs181848453", "rs138318162", "rs186913222", "rs141816674", "rs150801216"], AbstractString["19", "19", "19", "19", "19", "19", "19", "19", "19", "19"  …  "19", "19", "19", "19", "19", "19", "19", "19", "19", "19"], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [90974, 91106, 93542, 95981, 105021, 107866, 107894, 107958, 107962, 107987  …  59117949, 59117982, 59118028, 59118040, 59118044, 59118114, 59118148, 59118616, 59118779, 59118783], AbstractString["1", "1", "1", "1", "1", "1", "1", "1", "1", "1"  …  "1", "1", "1", 

Save bed and bim file

In [12]:
writeplink("SNP_data29a_missing", snpdata29c_missing)

Now we have the new `bed` and `bim` file.

In [13]:
;ls -l Ped29c.in SNP_data29a.bin SNP_def29a.* SNP_data29a_missing.*

Stacktrace:
 [1] depwarn(::String, ::Symbol) at /Applications/Julia-0.6.app/Contents/Resources/julia/lib/julia/sys.dylib:?
 [2] warn_shell_special(::String) at /Applications/Julia-0.6.app/Contents/Resources/julia/lib/julia/sys.dylib:?
 [3] #shell_parse#236(::String, ::Function, ::String, ::Bool) at /Applications/Julia-0.6.app/Contents/Resources/julia/lib/julia/sys.dylib:?
 [4] (::Base.#kw##shell_parse)(::Array{Any,1}, ::Base.#shell_parse, ::String, ::Bool) at /Applications/Julia-0.6.app/Contents/Resources/julia/lib/julia/sys.dylib:? (repeats 2 times)
 [5] @cmd(::ANY) at /Applications/Julia-0.6.app/Contents/Resources/julia/lib/julia/sys.dylib:?
 [6] include_string(::String, ::String) at /Applications/Julia-0.6.app/Contents/Resources/julia/lib/julia/sys.dylib:?
 [7] include_string(::Module, ::String, ::String) at /Users/huazhou/.julia/v0.6/Compat/src/Compat.jl:407
 [8] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/huazhou/.julia/v0.6/IJulia/src/execute_request.jl:154
 [9] eventlo

-rw-rw-r--  1 huazhou  staff     18868 Aug 21  2016 Ped29c.in
-rw-rw-r--  1 huazhou  staff  13416476 Aug 21  2016 SNP_data29a.bin
-rw-r--r--  1 huazhou  staff  13416483 Sep 12 09:41 SNP_data29a_missing.bed
-rw-r--r--  1 huazhou  staff   7282838 Sep 12 09:41 SNP_data29a_missing.bim
-rw-rw-r--  1 huazhou  staff  11990931 Aug 21  2016 SNP_def29a.in


For demonstration, we only need the `SNP_data29a.bed`, `SNP_def29a.in`, and `Ped29c.in` files.

In [14]:
# sanity check
snpbin29a_missing = SnpArray("SNP_data29a_missing"; people = 212, snps = 253141)
countnz(snpbin29a .≠ snpbin29a_missing) == sum(missings_by_snp)

INFO: v1.0 BED file detected


true